In [7]:
import numpy as np
import pandas as pd
import os
import re
from wordfreq import get_frequency_dict


### Pdf reading

In [5]:
%pip install --upgrade pdfplumber pdfminer -q

import pdfplumber

Note: you may need to restart the kernel to use updated packages.


In [6]:
def _convert_pdf_to_text(path):
    """Convert the PDF to text and store it in the self.content
    attribute.
    """
    text = []
    pdf = pdfplumber.open(path)
    for page in range(len(pdf.pages)):

        page_obj = pdf.pages[page]

        text.append(
            page_obj.extract_text(
                x_tolerance=1, x_tolerance_ratio=None, y_tolerance=1, layout=False, x_density=7.25, y_density=13, line_dir_render=None,keep_blank_chars=True,use_text_flow=True, char_dir_render=None, expand_ligatures=True
            )
        )
    text = "\n".join(text)
    
    return text

In [11]:

def process_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        text =  _convert_pdf_to_text(file)

        

        def pa(text):
            pattern_abstract = r'\s*a\s*b\s*s\s*t\s*r\s*a\s*c\s*t\s*'
            matches1 = re.findall(pattern_abstract, text)
            if matches1:
                match = ''.join(matches1[0])
                pattern_abstract = re.compile(pattern_abstract)
                return pattern_abstract.sub('\nabstract\n', text)
            else:
                return text

        def pi(text):
            pattern_intro = r'i\s*n\s*t\s*r\s*o\s*d\s*u\s*c\s*t\s*i\s*o\s*n'
            matches1 = re.findall(pattern_intro, text)
            if matches1:
                match = ''.join(matches1[0])
                pattern_intro = re.compile(pattern_intro)
                return pattern_intro.sub('\nintroduction\n', text)
            else:
                return text

        def pr(text):
            pattern_intro = r'r\s*e\s*f\s*e\s*r\s*e\s*n\s*c\s*e\s*s'
            matches1 = re.findall(pattern_intro, text)
            if matches1:
                match = ''.join(matches1[0])
                pattern_intro = re.compile(pattern_intro)
                return pattern_intro.sub('\nreferences\n', text)
            else:
                return text
            
        text = pa(text)
        text = pi(text)
        text = pr(text)
        text = text.lower()
        text = [[text]]
        # print(text)
        df = pd.DataFrame(text, columns=['raw_text'])
        # print(df['raw_text'][0])
        # df.head()
        text = text[0]

        def find_text_by_largest_font(page):
            largest_font_size = 0
            largest_font_chars = []

            for char in page.chars:
                size = char["size"]
                if size > largest_font_size:
                    largest_font_size = size
                    largest_font_chars = [char]
                elif size == largest_font_size:
                    largest_font_chars.append(char)

            return largest_font_chars

        with pdfplumber.open(file) as pdf:
            first_page = pdf.pages[0]
            largest_font_chars = find_text_by_largest_font(first_page)
            largest_font_text = ''.join(char['text'] for char in largest_font_chars)
        df['Title'] = largest_font_text


        word_prob = get_frequency_dict(lang='en', wordlist='large')
        max_word_len = max(map(len, word_prob))  # 34

        # def str_slice(text):
        def viterbi_segment(text, debug=False):
            probs, lasts = [1.0], [0]
            for i in range(1, len(text) + 1):
                new_probs = []
                for j in range(max(0, i - max_word_len), i):
                    substring = text[j:i]
                    length_reward = np.exp(len(substring))
                    freq = word_prob.get(substring, 0) * length_reward
                    compounded_prob = probs[j] * freq
                    new_probs.append((compounded_prob, j))
                    
                    if debug:
                        print(f'[{j}:{i}] = "{text[lasts[j]:j]} & {substring}" = ({probs[j]:.8f} & {freq:.8f}) = {compounded_prob:.8f}')

                prob_k, k = max(new_probs)  # max of a touple is the max across the first elements, which is the max of the compounded probabilities
                probs.append(prob_k)
                lasts.append(k)

                if debug:
                    print(f'i = {i}, prob_k = {prob_k:.8f}, k = {k}, ({text[k:i]})\n')


            # when text is a word that doesn't exist, the algorithm breaks it into individual letters.
            # in that case, return the original word instead
            if len(set(lasts)) == len(text):
                return text

            words = []
            k = len(text)
            while 0 < k:
                word = text[lasts[k]:k]
                words.append(word)
                k = lasts[k]
            words.reverse()
            return ' '.join(words)

        def split_message(message):
            new_message = ' '.join(viterbi_segment(wordmash, debug=False) for wordmash in message.split())
            return new_message

        def raw(text):
            # print("Input text:", text)  # Add this line for debugging
            if 'introduction' in text:
                new = text.split('introduction')
                # print("Split result:", new)  # Add this line for debugging
                return new[1] 
            else:
                return ''


        def header(text):
            if 'introduction' in text:
                new = text.split('introduction')
                print(new[0])
                return new[0]   
            else:
                return ''
            
        def abstract(text):
            new = text.split('abstract')
            if len(new) > 1:
                return new[1]
            else:
                return ''  
            
            

        def pc(data_text):
            data_text = data_text.split('\"' and "," and ":" and '\”' and '\“')
            text = ""
            for i in data_text:
                text = text + i
            print(len(text))

            text = ""

            for i in data_text:
                text = text + i
    
            return text
        
        mail_pattern = re.compile(r'\b\S+@\S+\.\S+\.?\S*\b')
        def remove_mail(text):
            return mail_pattern.sub('', text)
        
        url_pattern = r'https?://(?:www\.)?[^ ]+|www[^ ]+'

        def add_urls(text):
            httplinks = re.findall(url_pattern,text)
            return httplinks

        url_pattern = re.compile(url_pattern)
        
        def remove_urls(text):
            return url_pattern.sub('', text)
        
        def ref(text):
            if('references' in text):
                terms = text.split('references')
                return terms[1]
            else:
                return ''
        def wref(text):
            if('references' in text):
                terms = text.split('references')
                return terms[0]
            else:
                return text
            
        number_pattern = re.compile(r'\b(?<!\[)(-?\d+(?:\.\d+)?)\b(?![^\[]*\])')

        def remove_number(text):
            return number_pattern.sub('', text)


        def sen_seg(data_text):
            sentences = []

            sentence = ''

            # Iterate through the text
            for i in range(len(data_text)):
                char = data_text[i]
                # Append character to the current sentence
                sentence += char
                if char == '.':
                    # Check if the character before and after '.' are both numbers
                    if i > 0 and i < len(data_text) - 1 and data_text[i-1].isdigit() and data_text[i+1].isdigit():
                        continue  # Skip adding '.' to the sentence
                    else:
                        sentences.append(sentence.strip())
                        sentence = ''

            return sentences
        
        def checker(sentences):
            new = []
            for s in sentences:
                if(len(s) > 30):
                    new.append(s)
            return new

        
        def merge_sen(sentences):
            separator = " "
            clean = separator.join(sentences)
            return clean

        df['Title'] = df['Title'].apply(split_message)

        # print(df['raw_text'][0])
        df['header'] = df['raw_text'].apply(header)
        print(df['header'][0])
        df['abstract'] = df['header'].apply(abstract)
        df['raw_text'] = df['raw_text'].apply(raw)
        df['pc_removal'] = df['raw_text'].apply(pc)
        df['wmail'] = df['pc_removal'].apply(remove_mail)
        df['urls'] = df['wmail'].apply(add_urls)
        df['wurl'] = df['wmail'].apply(remove_urls)
        df['refs'] = df['wurl'].apply(ref)
        df['wrefs'] = df['wurl'].apply(wref)
        df['wnumbers'] = df['wrefs']
        df['sentences'] = df['wnumbers'].apply(sen_seg)
        df['sentences'] = df['sentences'].apply(checker)
        df['clean'] = df['sentences'].apply(merge_sen)
   
    return df


def process_directory(directory):
    
    combined_df = pd.DataFrame() 

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.pdf'):
                pdf_path = os.path.join(root, file)
                df = process_pdf(pdf_path) 
                # print(df['clean'][0])
                combined_df = pd.concat([combined_df, df])

        for dir in dirs:
            subdir = os.path.join(root, dir)
            process_directory(subdir)

    return combined_df

pdf_directory = './../Createdb/docs'

if os.path.isdir(pdf_directory):
    combined_df = process_directory(pdf_directory)
    # Check if the combined DataFrame is not empty
    if not combined_df.empty:
        # Export the combined DataFrame to CSV
        combined_df.to_csv("combined_data.csv", index=False)
    else:
        print("Combined DataFrame is empty.")
else:
    if pdf_directory.endswith('.pdf'):
        df = process_pdf(pdf_directory)  
        # Check if the DataFrame is not empty
        if not df.empty:
            # Export the DataFrame to CSV
            df.to_csv("combined_data.csv", index=False)
            
        else:
            print("DataFrame is empty.")
    else:
        print("Invalid path. Please provide a directory or a PDF file.")

http://ammtiac.alionscience.com the ammtiac quarterly, volume 3, number 4 3

http://ammtiac.alionscience.com the ammtiac quarterly, volume 3, number 4 3

23667
structural damage identification in laminated structures
using frf data
j.v. arau´ jo dos santos, c.m. mota soares *, c.a. mota soares, n.m.m. maia
idmec/ist, instituto superior te´cnico, av. rovisco pais, 1049-001 lisbon, portugal
available online 5 november 2004
abstract
a damage identification technique based on frequency response functions (frf) sensitivities is presented. this technique leads to
a set of linear equations, which is solved using an algorithm that constrains the solution to be physically admissible. damage sim-
ulation and identification on a laminated rectangular plate is performed. the influence of the number of natural frequencies and
mode shapes used on the frf computation, as well as the frequency range, the excitation location and the number of measured
degrees of freedom (m-dof) is studied. numerical te

In [10]:
combined_df


,raw_text,Title,header,abstract,pc_removal,wmail,urls,wurl,refs,wrefs,wnumbers,sentences,clean
0,\nthe subject of structural health monitoring ...,http : / / a m m t i a c . a l i o n s c i e n...,http://ammtiac.alionscience.com the ammtiac qu...,,\nthe subject of structural health monitoring ...,\nthe subject of structural health monitoring ...,[http://ammtiac.alionscience.com/quarterly\nht...,\nthe subject of structural health monitoring ...,"\n[1] structural health monitoring workshop, s...",\nthe subject of structural health monitoring ...,\nthe subject of structural health monitoring ...,[the subject of structural health monitoring (...,the subject of structural health monitoring (s...
0,\nthe damage identification in composites lami...,Structuraldamageidentiﬁcationinlaminatedstruct...,structural damage identification in laminated ...,\na damage identification technique based on f...,\nthe damage identification in composites lami...,\nthe damage identification in composites lami...,[www.elsevier.com/locate/compstruct\nto],\nthe damage identification in composites lami...,"\n[1] doebling sw, farrar cr, prime mb, shevit...",\nthe damage identification in composites lami...,\nthe damage identification in composites lami...,[the damage identification in composites lamin...,the damage identification in composites lamina...
0,\nstructural optimization has become one of th...,JournalofConstructionalSteelResearch,journal of constructional steel research 65 (2...,"\nin this paper, a particle swarm optimizer wi...",\nstructural optimization has become one of th...,\nstructural optimization has become one of th...,[],\nstructural optimization has become one of th...,"\n[1] kaveh a, farahmand azar b, talatahari s....",\nstructural optimization has become one of th...,\nstructural optimization has become one of th...,[structural optimization has become one of the...,structural optimization has become one of the ...
